<a href="https://colab.research.google.com/github/deepavasanthkumar/deepcodesnippets/blob/master/spark_date_time_patterns.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317145 sha256=cb9f51805f33ea5259bf82b14c9c9a330446845dfd4d560790201edcafb09284
  Stored in directory: /root/.cache/pip/wheels/9f/34/a4/159aa12d0a510d5ff7c8f0220abbea42e5d81ecf588c4fd884
Successfully built pyspark


Using dataset available from https://www.kaggle.com/datasets/sumanthvrao/daily-climate-time-series-data?resource=download

In [3]:
from google.colab import files
uploaded = files.upload()

Saving DailyDelhiClimateTrain.csv to DailyDelhiClimateTrain.csv


In [2]:
from pyspark.sql import SparkSession
  
spark = SparkSession.builder.appName("DataFrame").getOrCreate()

In [5]:
df = spark.read.csv('DailyDelhiClimateTrain.csv', inferSchema=True, header=True, sep = ",")

df.printSchema()

root
 |-- date: date (nullable = true)
 |-- meantemp: double (nullable = true)
 |-- humidity: double (nullable = true)
 |-- wind_speed: double (nullable = true)
 |-- meanpressure: double (nullable = true)



#current_date

In [6]:
from pyspark.sql.functions import *

In [8]:
df = df.withColumn("current_date", current_date())
df.show(2)

+----------+--------+--------+----------+------------------+------------+
|      date|meantemp|humidity|wind_speed|      meanpressure|current_date|
+----------+--------+--------+----------+------------------+------------+
|2013-01-01|    10.0|    84.5|       0.0|1015.6666666666666|  2023-04-25|
|2013-01-02|     7.4|    92.0|      2.98|            1017.8|  2023-04-25|
+----------+--------+--------+----------+------------------+------------+
only showing top 2 rows



#current_timestamp

In [12]:
df = df.withColumn("current_ts", current_timestamp())
df.show(2, True)

+----------+--------+--------+----------+------------------+------------+--------------------+
|      date|meantemp|humidity|wind_speed|      meanpressure|current_date|          current_ts|
+----------+--------+--------+----------+------------------+------------+--------------------+
|2013-01-01|    10.0|    84.5|       0.0|1015.6666666666666|  2023-04-25|2023-04-25 06:09:...|
|2013-01-02|     7.4|    92.0|      2.98|            1017.8|  2023-04-25|2023-04-25 06:09:...|
+----------+--------+--------+----------+------------------+------------+--------------------+
only showing top 2 rows



#date_format

In [13]:
df = df.withColumn("formattedDate", date_format("current_date", "dd/MMM/yyyy"))
df.show(2)

+----------+--------+--------+----------+------------------+------------+--------------------+-------------+
|      date|meantemp|humidity|wind_speed|      meanpressure|current_date|          current_ts|formattedDate|
+----------+--------+--------+----------+------------------+------------+--------------------+-------------+
|2013-01-01|    10.0|    84.5|       0.0|1015.6666666666666|  2023-04-25|2023-04-25 06:11:...|  25/Apr/2023|
|2013-01-02|     7.4|    92.0|      2.98|            1017.8|  2023-04-25|2023-04-25 06:11:...|  25/Apr/2023|
+----------+--------+--------+----------+------------------+------------+--------------------+-------------+
only showing top 2 rows



#Truncate dates

In [21]:
df = df.withColumn("year_trunc", date_trunc("year", "date"))
df.select(col("year_trunc"), col("date")).show(15)

+-------------------+----------+
|         year_trunc|      date|
+-------------------+----------+
|2013-01-01 00:00:00|2013-01-01|
|2013-01-01 00:00:00|2013-01-02|
|2013-01-01 00:00:00|2013-01-03|
|2013-01-01 00:00:00|2013-01-04|
|2013-01-01 00:00:00|2013-01-05|
|2013-01-01 00:00:00|2013-01-06|
|2013-01-01 00:00:00|2013-01-07|
|2013-01-01 00:00:00|2013-01-08|
|2013-01-01 00:00:00|2013-01-09|
|2013-01-01 00:00:00|2013-01-10|
|2013-01-01 00:00:00|2013-01-11|
|2013-01-01 00:00:00|2013-01-12|
|2013-01-01 00:00:00|2013-01-13|
|2013-01-01 00:00:00|2013-01-14|
|2013-01-01 00:00:00|2013-01-15|
+-------------------+----------+
only showing top 15 rows



#from_utc_timestamp

In [22]:
df = df.withColumn("et_ts", from_utc_timestamp("current_timestamp", "EST"))
df.select(col("current_timestamp"), col("et_ts")).show(15)

+--------------------+--------------------+
| current_timestamp()|               et_ts|
+--------------------+--------------------+
|2023-04-25 06:19:...|2023-04-25 01:19:...|
|2023-04-25 06:19:...|2023-04-25 01:19:...|
|2023-04-25 06:19:...|2023-04-25 01:19:...|
|2023-04-25 06:19:...|2023-04-25 01:19:...|
|2023-04-25 06:19:...|2023-04-25 01:19:...|
|2023-04-25 06:19:...|2023-04-25 01:19:...|
|2023-04-25 06:19:...|2023-04-25 01:19:...|
|2023-04-25 06:19:...|2023-04-25 01:19:...|
|2023-04-25 06:19:...|2023-04-25 01:19:...|
|2023-04-25 06:19:...|2023-04-25 01:19:...|
|2023-04-25 06:19:...|2023-04-25 01:19:...|
|2023-04-25 06:19:...|2023-04-25 01:19:...|
|2023-04-25 06:19:...|2023-04-25 01:19:...|
|2023-04-25 06:19:...|2023-04-25 01:19:...|
|2023-04-25 06:19:...|2023-04-25 01:19:...|
+--------------------+--------------------+
only showing top 15 rows



#to_utc_timestamp

In [25]:
df = df.withColumn("utc_ts", to_utc_timestamp("et_ts", "EST"))
df.select(col("current_timestamp"), col("et_ts"), col("utc_ts")).show(15)


+--------------------+--------------------+--------------------+
| current_timestamp()|               et_ts|              utc_ts|
+--------------------+--------------------+--------------------+
|2023-04-25 06:35:...|2023-04-25 01:35:...|2023-04-25 06:35:...|
|2023-04-25 06:35:...|2023-04-25 01:35:...|2023-04-25 06:35:...|
|2023-04-25 06:35:...|2023-04-25 01:35:...|2023-04-25 06:35:...|
|2023-04-25 06:35:...|2023-04-25 01:35:...|2023-04-25 06:35:...|
|2023-04-25 06:35:...|2023-04-25 01:35:...|2023-04-25 06:35:...|
|2023-04-25 06:35:...|2023-04-25 01:35:...|2023-04-25 06:35:...|
|2023-04-25 06:35:...|2023-04-25 01:35:...|2023-04-25 06:35:...|
|2023-04-25 06:35:...|2023-04-25 01:35:...|2023-04-25 06:35:...|
|2023-04-25 06:35:...|2023-04-25 01:35:...|2023-04-25 06:35:...|
|2023-04-25 06:35:...|2023-04-25 01:35:...|2023-04-25 06:35:...|
|2023-04-25 06:35:...|2023-04-25 01:35:...|2023-04-25 06:35:...|
|2023-04-25 06:35:...|2023-04-25 01:35:...|2023-04-25 06:35:...|
|2023-04-25 06:35:...|202